## 목표

- 음성 데이터에 변형을 가하고 KFold를 이용해 다양한 학습을 하게 하여 성능을 높인다
- CNN 모델과 Kobert-Transformers로 각각 특징을 추출하여 소프트보팅 앙상블 모델을 만든다

### 순서
- 1. 필요 모듈들 설치 및 임포트
- 2. 기본 데이터 불러오기
- 3. 필요한 함수들 설정(모델 포함)
- 4. 모델 매개변수 설정
- 5. 모델학습
- 6. 평가
- 7. 예측

#### 1. 필요 모듈들 설치 및 임포트

In [1]:
import librosa
# import librosa.display
# import IPython.display as ipd
# from IPython.display import Audio

# from random import randint
import numpy as np
import pandas as pd
# import pydub #오디오 파일을 다루는 라이브러리 : 변환 조작 재생 분석 등
# import gTTS # 텍스트를 음성으로 변환하는 라이브러리. 나는 당장 쓸 일 없을 듯?

from tqdm import tqdm
# import np_utils
# import matplotlib.pyplot as plt
# import seaborn as sns

# import nlpaug.augmenter.audio as naa
# import wave

import glob
import os
import json
# import shutil
# import sys
# import logging
# import unicodedata
# from shutil import copyfile
# import warnings
# warnings.filterwarnings("always")


# from sklearn.preprocessing import LabelEncoder,StandardScaler, OneHotEncoder,  MinMaxScaler, RobustScaler
# from sklearn.metrics import confusion_matrix, classification_report
# from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, GridSearchCV
# from sklearn.metrics import accuracy_score, f1_score
# from sklearn.svm import SVC 
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.pipeline import Pipeline

# import joblib
# from joblib import dump, load
# from xgboost import XGBClassifier 

# import keras
# from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping, Callback
# from keras.models import Sequential
# from tensorflow.keras.layers import InputLayer, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, LSTM, concatenate
# from tensorflow.keras.optimizers import Adam
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils import to_categorical

# import tensorflow as tf
# from sentence_transformers import SentenceTransformer

# import torch
# from torch import nn
# import torch.optim as optim
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# from kobert_transformers import get_kobert_model, get_distilkobert_model, get_tokenizer

# np.bool = np.bool_

# import mxnet
# import gluonnlp as nlp

# from transformers import AdamW
# from transformers.optimization import get_cosine_schedule_with_warmup
# from transformers import BertModel

from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift # 음성 데이터 증강용

In [75]:
!pip install kobert-transformers
!pip install audiomentations

C:\Users\smhrd\anaconda3\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=6>
  return process_handler(cmd, _system_body)
C:\Users\smhrd\anaconda3\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=7>
  return process_handler(cmd, _system_body)
C:\Users\smhrd\anaconda3\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=8>
  return process_handler(cmd, _system_body)


   ---------------------------------------- 0.0/82.3 kB ? eta -:--:--
   -------------- ------------------------- 30.7/82.3 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 82.3/82.3 kB 1.1 MB/s eta 0:00:00


C:\Users\smhrd\anaconda3\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=6>
  return process_handler(cmd, _system_body)
C:\Users\smhrd\anaconda3\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=7>
  return process_handler(cmd, _system_body)
C:\Users\smhrd\anaconda3\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=8>
  return process_handler(cmd, _system_body)


#### 2. 기본 데이터 불러오기

In [2]:
train_csv = pd.read_csv('e:/Data2/csv/train.csv')
test_csv = pd.read_csv('e:/Data2/csv/test.csv')
exam_list = ['5fbe364744697678c497c07f','5fbe361844697678c497c07e']

train_audio_path = 'e:/Data2/train/'
test_audio_path = 'e:/Data2/valid/'
exam_audio_path = 'e:/Data2/exam/'

# X = np.load('./features/features5.npy')
y = train_csv.iloc[:,-7:]

#### 3. 필요한 함수들 설정(모델 포함)
- 비프음 함수
- 음성 특징 추출 함수
- 음성 특징 학습 함수(CNN)
- 텍스트 임베딩 함수(KoBERT)


In [3]:
#에러 발생 혹은 작업 완료시 beep로 알리기
def beepError():
    fr = 1000    # range : 37 ~ 32767
    du = 2000     # 1000 ms ==1second
    ws.Beep(fr, du) # winsound.Beep(frequency, duration)

def beepSuccess():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    ws.Beep(fr, du) # winsound.Beep(frequency, duration)

In [4]:
def perceptual_sharpness(audio_path, sr=16000, n_fft=400, hop_length=160):
    # 음원 파일 로드
    y= audio_path
    sr = sr

    # STFT 수행
    D = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))

    # 주파수 대역별로 에너지 계산
    energy = np.sum(D, axis=0)

    # 고주파수 대역 성분 추출
    high_freq_energy = energy[3000:8000]  # 예시로 3000Hz에서 6000Hz 사이의 주파수 대역을 고주파수 대역으로 설정

    # Perceptual Sharpness 계산
    sharpness = np.sum(np.log1p(high_freq_energy))

    return sharpness

In [5]:
def extract_feature(audio_path, csv_file):
    feature_list = []
#     file_array = np.array(csv_file['wav_id'])
    file_array = np.array(csv_file) #테스트용

    for i, file_name in tqdm(enumerate(file_array)):

        audio, sr = librosa.load(os.path.join(audio_path, f'{file_name}.wav'), sr=16000)
         ##Mel-spectrogram 구현
        spectrogram = librosa.stft(audio, n_fft=400, hop_length= 160) 
        power_spectrogram = spectrogram**2
        mel = librosa.feature.melspectrogram(S=power_spectrogram, sr=sr)
        mel = librosa.power_to_db(np.abs(mel)**2)
        #mfcc 구현
        mfccs = librosa.feature.mfcc(S = mel, n_mfcc=100)
        stft = np.abs(spectrogram)
        chroma_stft = librosa.feature.chroma_stft(S=stft,hop_length=160)
        rms = librosa.feature.rms(y=audio)
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
        spectral_bandwidths = librosa.feature.spectral_bandwidth(y=audio, sr=sr)
        spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)
        zero_crossing_rates = librosa.feature.zero_crossing_rate(y=audio)
        chroma_cens = librosa.feature.chroma_cens(C=spectrogram, sr=sr)
        tempo, _ = librosa.beat.beat_track(y=audio, sr=sr)
        ps = perceptual_sharpness(audio)

        try:
            mfccs_mean = mfccs.mean(axis=1)
            mfccs_var = mfccs.mean(axis=1)

            for k in range(len(mfccs_mean)):
                locals()[f'mfccs_mean_{k}'] = mfccs_mean[k]
                locals()[f'mfccs_var_{k}'] = mfccs_var[k]
                chroma_stft_mean = chroma_stft.mean()
                chroma_stft_var = chroma_stft.var()
                rms_mean = rms.mean()
                rms_var = rms.var()
                spectral_centroids_mean = spectral_centroids.mean()
                spectral_centroids_var = spectral_centroids.var()
                spectral_bandwidths_mean = spectral_bandwidths.mean()
                spectral_bandwidths_var = spectral_bandwidths.var()
                spectral_rolloff_mean = spectral_rolloff.mean()
                spectral_rolloff_var = spectral_rolloff.var()
                zero_crossing_rates_mean = zero_crossing_rates.mean()
                zero_crossing_rates_var = zero_crossing_rates.var()
                harmony_mean = chroma_cens.mean()
                harmony_var = chroma_cens.var()
                tempo_mean = tempo.mean()
                tempo_var = tempo.var()
                perceptual_sharpness_mean = ps.mean()
                perceptual_sharpness_var = ps.var()
        except Exception as e:
            print(f'{i}번째 파일에서 문제 발생',e)
            continue

        #합치기
        features = np.array([])
        for j in range(len(mfccs_mean)):
            features = np.hstack((features,locals()[f'mfccs_mean_{j}'],locals()[f'mfccs_var_{j}']))

        features = np.hstack((features,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroids_mean
                             ,spectral_centroids_var, spectral_bandwidths_mean, spectral_bandwidths_var,spectral_rolloff_mean
                             , spectral_rolloff_var,zero_crossing_rates_mean, zero_crossing_rates_var, harmony_mean, harmony_var
                             , tempo_mean,tempo_var,perceptual_sharpness_mean,perceptual_sharpness_var))
        augmenter = naa.NoiseAug() #데이터 증강을 위해 노이즈된 특성 추가
        augmented_features = np.squeeze(augmenter.augment(features))
        features =np.hstack((features,augmented_features))

        feature_list.append(features)
    
    
    return feature_array

In [62]:
def extract_feature_2(audio_path, csv_file):
    feature_list = []
    file_array = np.array(csv_file['wav_id'])
#     file_array = np.array(csv_file) #테스트용
    augment = Compose([
        AddGaussianNoise(p=0.5),
        TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
        PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
        Shift(p=0.5)
        ])
    for i, file_name in tqdm(enumerate(file_array)):

        audio, sr = librosa.load(os.path.join(audio_path, f'{file_name}.wav'), sr=16000)
        augmented_audio = augment(samples=audio, sample_rate=sr)
         ##Mel-spectrogram 구현
        spectrogram = librosa.stft(audio, n_fft=400, hop_length= 160) 
        power_spectrogram = spectrogram**2
        mel = librosa.feature.melspectrogram(S=power_spectrogram, sr=sr)
        mel = librosa.power_to_db(np.abs(mel)**2)
        #mfcc 구현
        mfccs = librosa.feature.mfcc(S = mel, n_mfcc=100)
        stft = np.abs(spectrogram)
        chroma_stft = librosa.feature.chroma_stft(S=stft,hop_length=160)
        rms = librosa.feature.rms(y=audio)
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
        spectral_bandwidths = librosa.feature.spectral_bandwidth(y=audio, sr=sr)
        spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)
        zero_crossing_rates = librosa.feature.zero_crossing_rate(y=audio)
        chroma_cens = librosa.feature.chroma_cens(C=spectrogram, sr=sr)
        tempo, _ = librosa.beat.beat_track(y=audio, sr=sr)
        ps = perceptual_sharpness(audio)

        try:
            mfccs_mean = mfccs.mean(axis=1)
            mfccs_var = mfccs.mean(axis=1)

            for k in range(len(mfccs_mean)):
                locals()[f'mfccs_mean_{k}'] = mfccs_mean[k]
                locals()[f'mfccs_var_{k}'] = mfccs_var[k]
                chroma_stft_mean = chroma_stft.mean()
                chroma_stft_var = chroma_stft.var()
                rms_mean = rms.mean()
                rms_var = rms.var()
                spectral_centroids_mean = spectral_centroids.mean()
                spectral_centroids_var = spectral_centroids.var()
                spectral_bandwidths_mean = spectral_bandwidths.mean()
                spectral_bandwidths_var = spectral_bandwidths.var()
                spectral_rolloff_mean = spectral_rolloff.mean()
                spectral_rolloff_var = spectral_rolloff.var()
                zero_crossing_rates_mean = zero_crossing_rates.mean()
                zero_crossing_rates_var = zero_crossing_rates.var()
                harmony_mean = chroma_cens.mean()
                harmony_var = chroma_cens.var()
                tempo_mean = tempo.mean()
                tempo_var = tempo.var()
                perceptual_sharpness_mean = ps.mean()
                perceptual_sharpness_var = ps.var()
        except Exception as e:
            print(f'{i}번째 파일에서 문제 발생',e)
            continue

        #합치기
        features = np.array([])
        for j in range(len(mfccs_mean)):
            features = np.hstack((features,locals()[f'mfccs_mean_{j}'],locals()[f'mfccs_var_{j}']))

        features = np.hstack((features,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroids_mean
                             ,spectral_centroids_var, spectral_bandwidths_mean, spectral_bandwidths_var,spectral_rolloff_mean
                             , spectral_rolloff_var,zero_crossing_rates_mean, zero_crossing_rates_var, harmony_mean, harmony_var
                             , tempo_mean,tempo_var,perceptual_sharpness_mean,perceptual_sharpness_var))
        # Audiomentations 증강 파이프라인 설정
        spectrogram = librosa.stft(augmented_audio, n_fft=400, hop_length= 160) 
        power_spectrogram = spectrogram**2
        mel = librosa.feature.melspectrogram(S=power_spectrogram, sr=sr)
        mel = librosa.power_to_db(np.abs(mel)**2)
        #mfcc 구현
        mfccs = librosa.feature.mfcc(S = mel, n_mfcc=100)
        stft = np.abs(spectrogram)
        chroma_stft = librosa.feature.chroma_stft(S=stft,hop_length=160)
        rms = librosa.feature.rms(y=augmented_audio)
        spectral_centroids = librosa.feature.spectral_centroid(y=augmented_audio, sr=sr)
        spectral_bandwidths = librosa.feature.spectral_bandwidth(y=augmented_audio, sr=sr)
        spectral_rolloff = librosa.feature.spectral_rolloff(y=augmented_audio, sr=sr)
        zero_crossing_rates = librosa.feature.zero_crossing_rate(y=augmented_audio)
        chroma_cens = librosa.feature.chroma_cens(C=spectrogram, sr=sr)
        tempo, _ = librosa.beat.beat_track(y=augmented_audio, sr=sr)
        ps = perceptual_sharpness(augmented_audio)

        try:
            mfccs_mean = mfccs.mean(axis=1)
            mfccs_var = mfccs.mean(axis=1)

            for k in range(len(mfccs_mean)):
                locals()[f'mfccs_mean_{k}'] = mfccs_mean[k]
                locals()[f'mfccs_var_{k}'] = mfccs_var[k]
                chroma_stft_mean = chroma_stft.mean()
                chroma_stft_var = chroma_stft.var()
                rms_mean = rms.mean()
                rms_var = rms.var()
                spectral_centroids_mean = spectral_centroids.mean()
                spectral_centroids_var = spectral_centroids.var()
                spectral_bandwidths_mean = spectral_bandwidths.mean()
                spectral_bandwidths_var = spectral_bandwidths.var()
                spectral_rolloff_mean = spectral_rolloff.mean()
                spectral_rolloff_var = spectral_rolloff.var()
                zero_crossing_rates_mean = zero_crossing_rates.mean()
                zero_crossing_rates_var = zero_crossing_rates.var()
                harmony_mean = chroma_cens.mean()
                harmony_var = chroma_cens.var()
                tempo_mean = tempo.mean()
                tempo_var = tempo.var()
                perceptual_sharpness_mean = ps.mean()
                perceptual_sharpness_var = ps.var()
        except Exception as e:
            print(f'{i}번째 파일에서 문제 발생',e)
            continue

        #합치기
        augmented_features = np.array([])
        for j in range(len(mfccs_mean)):
            augmented_features = np.hstack((augmented_features,locals()[f'mfccs_mean_{j}'],locals()[f'mfccs_var_{j}']))

        augmented_features = np.hstack((augmented_features,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroids_mean
                             ,spectral_centroids_var, spectral_bandwidths_mean, spectral_bandwidths_var,spectral_rolloff_mean
                             , spectral_rolloff_var,zero_crossing_rates_mean, zero_crossing_rates_var, harmony_mean, harmony_var
                             , tempo_mean,tempo_var,perceptual_sharpness_mean,perceptual_sharpness_var))
        features =np.hstack((features,augmented_features))

        feature_list.append(features)
        # if i>0 and i%100 == 0:
        #     print(f'{i}번째 파일 완료')
        #     locals()[f'X_{int(i/100)}'] = np.array(feature_list).reshape(-1,len(feature_list[0]))
    feature_array  = np.array(feature_list).reshape(-1,len(feature_list[0]))

    return feature_array

In [6]:
au'5fbe364744697678c497c07f'

['5fbe364744697678c497c07f', '5fbe361844697678c497c07e']

In [ ]:
# features = extract_feature(test_audio_path,test_list) #위 함수에서 test용으로 변경하고 실행해봄
# features.shape
features = extract_feature(train_audio_path,train_csv) 

In [ ]:
np.save('./features/train_features_6',features) # 6번째 train_features. 스케일링 안 된 것. 나중에 학습할 때 스케일링 fit 해주자.

In [ ]:
val_features = extract_feature(valid_audio_path,valid_csv)

In [ ]:
np.save('./features/valid_features_6',val_features) # valid features는 사실 두 번째지만 train_features랑 번호를 맞추기 위해서. 이것도 나중에 스케일러로 트랜스폼해 줘야.

In [ ]:
features = extract_feature_2(train_audio_path,train_csv) 

#3395번째 파일에서 문제 발생...?

In [60]:
print(feature_array[0][0:5])
print(val_features[3395][0:5])

# val_features= np.insert(val_features,3395,feature_array,axis=0)
# np.save('./features/train_features_7',val_features) # 해당 파일 특징 개별 추출해서 삽입

[-661.94946289+0.j -661.94946289+0.j    5.81620216+0.j    5.81620216+0.j
    5.66127253+0.j]
[-661.94946289+0.j -661.94946289+0.j    5.81620216+0.j    5.81620216+0.j
    5.66127253+0.j]


In [102]:
np.save('./features/train_features_7',features) # 스케일링 안 됨

In [57]:
features.shape

(29319, 436)

In [ ]:
val_features = extract_feature_2(valid_audio_path,valid_csv)

11138it [43:59,  5.37it/s]

In [ ]:
np.save('./features/valid_features_7',val_features) #스케일링 안 함

In [58]:
val_features.shape

(29319, 436)

In [ ]:
def cnn_model(data):
    model=Sequential()
    model.add(InputLayer(shape=(data.shape[1],data.shape[2])))
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(units=7, activation='softmax'))
    model.compile(optimizer = 'adamW' , loss = 'categorical_crossentropy' , metrics = ['accuracy',recall,precision]) #metrics 추가

    return model

### 결과
- 이 파일과는 별도로 Colab에서 KoBERT를 실행해 보려고 했으나 자꾸 import 관련해서 문제가 생김
- 작동하는 데 문제가 없고 KoBERT보다 감정분석에서 뛰어난 성능을 보이는 KoELECTRA로 대체하기로 함(9차 시도)